# 几何数据集测试
* 测试数据集由 mylibs.data.gen_gemotry模块生成

#### 数据集说明
    数据集由以下几何图形构成：直线，圆，椭圆，多边形
    
#### 实验目的
    观察MLP网络对几何数据集的分类性能
    
#### 实验步骤
    * 构造几何数据集
    * 构建MLP网络
    * 网络训练
    * 网路预测
    
#### 实验结果
    ased

### 构造数据集
    由lab_gemotry-data.ipynb生成.

In [1]:
%matplotlib inline
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

Using TensorFlow backend.


In [2]:
import os

data_path='%s/work/data/gemotry'%os.getenv('HOME')#数据目录
train_dir='%s/train'%data_path  #训练目录
valid_dir='%s/valid'%data_path  #校验目录
test_dir ='%s/test'%data_path   #测试目录
out_dir='%s/work/data/labs_out/lab_geometry-cnn'%os.getenv('HOME')   #输出目录
log_dir='%s/log_dir'%out_dir    #输出日志目录
cp_file='%s/cp_file.h5'%out_dir #训练断点
model_file='%s/model.h5'%out_dir#模型文件

input_shape=(224,224,3)
target_size=(224,224)
epochs=3
num_class=10
batch_size=32

# 数据生成器


In [3]:
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K

#构造图像数据生成器:train
gen_train = ImageDataGenerator(
        featurewise_center           =False,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.,
        width_shift_range            =0.,
        height_shift_range           =0.,
        shear_range                  =0.,
        zoom_range                   =0.,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =False,
        vertical_flip                =False,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_train=gen_train.flow_from_directory(directory='%s/train'%(data_path)
                                         ,batch_size=batch_size
                                         ,target_size=target_size)
#构造图像数据生成器:valid
gen_valid = ImageDataGenerator(
        featurewise_center           =False,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.,
        width_shift_range            =0.,
        height_shift_range           =0.,
        shear_range                  =0.,
        zoom_range                   =0.,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =False,
        vertical_flip                =False,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_valid=gen_valid.flow_from_directory(directory='%s/valid'%(data_path)
                                         ,batch_size=batch_size
                                         ,target_size=target_size)

#构造图像数据生成器:test
gen_test = ImageDataGenerator(
        featurewise_center           =False,
        samplewise_center            =False,
        featurewise_std_normalization=False,
        samplewise_std_normalization =False,
        zca_whitening                =False,
        zca_epsilon                  =1e-6,
        rotation_range               =0.,
        width_shift_range            =0.,
        height_shift_range           =0.,
        shear_range                  =0.,
        zoom_range                   =0.,
        channel_shift_range          =0.,
        fill_mode                    ='nearest',
        cval                         =0.,
        horizontal_flip              =False,
        vertical_flip                =False,
        rescale                      =1./255,
        preprocessing_function       =None,
        data_format                  =K.image_data_format()
       )
data_test=gen_test.flow_from_directory(directory='%s/test'%(data_path)
                                       ,batch_size=batch_size
                                       ,shuffle=False                                       
                                       ,target_size=target_size)

Found 20000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [4]:
#data=next(data_train)
#print(data[1])

### 模型创建

In [5]:
from keras import models,layers,optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=input_shape,name='conv2d_1'))
model.add(layers.MaxPooling2D((2, 2),name='max_pooling2d_1'))
model.add(layers.Flatten(name='flatten_1'))
model.add(layers.Dense(512, activation='relu',name='dense_1'))
model.add(layers.Dense(num_class, activation='softmax',name='dense_2'))

#打印模型
model.summary()

#模型编译
model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.RMSprop(lr=1e-4),
          metrics=['acc'])    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 394272)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               201867776 
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 201,873,802
Trainable params: 201,873,802
Non-trainable params: 0
_________________________________________________________________


## 模型训练

In [6]:
#模型训练
print('training beginning ......')

#断点加载
if os.path.exists(cp_file):
    print('load check point:',cp_file)
    model.load_weights(cp_file)
            
#断点训练:monitor监控参数可以通过score = model.evaluate(x_test, y_test, verbose=0)的score查询
checkpoint_cb = keras.callbacks.ModelCheckpoint(cp_file, monitor='val_acc', verbose=1, save_best_only=True, mode='auto',period=2)
#EarlyStopping
earlyStopping_cb=keras.callbacks.EarlyStopping(monitor='acc', patience=3, verbose=0, mode='max')
#TensorBoard
tensorBoard_cb=keras.callbacks.TensorBoard(log_dir=log_dir)
#回调函数序列
callbacks_list = [checkpoint_cb,earlyStopping_cb,tensorBoard_cb]

#模型训练
history = model.fit_generator(
  data_train,
  steps_per_epoch=np.ceil(data_train.samples/batch_size),
  epochs=epochs,
  validation_data=data_valid,
  validation_steps=50,
  callbacks=callbacks_list)
print('history:',history.history)

#保存模型
print('save model file:',model_file)
model.save(model_file)

training beginning ......
load check point: /home/hjw/work/data/labs_out/lab_geometry-cnn/cp_file.h5
Epoch 1/3
  8/625 [..............................] - ETA: 17:21 - loss: 0.1051 - acc: 0.9727

KeyboardInterrupt: 

## 模型测试

In [ ]:
#模型测试
#计算精度
def compute_acc(y_pred,y_true):
    acc=(y_pred-y_true)==0
    return acc.sum()/acc.size

print('predicting beginning ......')
#type(y_pred)=> <class 'numpy.ndarray'>
y_pred=model.predict_generator(
    data_test, 
    steps=None, #预测轮数
    max_queue_size=32, 
    workers=1, 
    use_multiprocessing=False, 
    verbose=1)

acc=compute_acc(np.argmax(y_pred,axis=1),data_test.classes)
print('samples:',data_test.samples)
print('classes[:2]:')
print(data_test.classes[:2])
print('y_pred.shape:',y_pred.shape)
print('y_pred[:2]:')
print(y_pred[:2])
print('准确率:',acc)